# Lottery Scheduler Simulator

## Introduction

This notebooks implements a simple lottery scheduler simulator.  By changing the *number of time steps*, the *set of processes*, and the *number of tickets per process* (all on lines 14 through 18), one can explore how various settings for those parameters affect the generated schedules.

## Running a cell
If you click on the cell below, you can run it by hitting the *Play* button to the left of it.

## Output
The output that is generated from the cell is a graph containing *time steps* on the x-axis, *percentage of CPU use* on the y-axis, with a line plot for each process.

In [ ]:
def get_associated_process(ticket_bounds_arg, lottery_value_arg):
    for index in range(len(ticket_bounds_arg)):
        if (lottery_value_arg < ticket_bounds_arg[index]):
            return index
    return -1

# Need the 'random' package to support lottery scheduling
import random
import copy
import matplotlib.pyplot as plotter
random.seed(525)

# Number of timesteps to emulate in scheduling
number_of_timesteps = 1000

# Each process has a name and an associated number of tickets 
process_names = ["A", "B", "C"]
process_tickets = [100,100,100]


# Summary information about processes
number_of_processes = len(process_names)
number_of_tickets = 0
ticket_bounds = [0] * number_of_processes
for process in range(number_of_processes):
    number_of_tickets = number_of_tickets + process_tickets[process]
    ticket_bounds[process] = number_of_tickets

print("Number of processes: ", number_of_processes)
print("Number of tickets: ", number_of_tickets)
print("Simulating ", number_of_timesteps, " timesteps")
# This is the "bounds array"
print(ticket_bounds)

# Schedule is initially empty
schedule = [""] * number_of_timesteps

# For each timestep, choose a random ticket and find the associated process
# Record in schedule
for timestep in range(number_of_timesteps):
    lottery_value = random.randint(0,number_of_tickets)
    chosen_process = get_associated_process(ticket_bounds,lottery_value)
    # print(timestep," ", chosen_process, " ", lottery_value)
    schedule[timestep] = chosen_process

# print("Schedule generated:")
# for timestep in range(number_of_timesteps):
#     print(timestep," ", schedule[timestep])

# Summarize the schedule to graph it
schedule_counts = []
schedule_percentages = []
for process in range(number_of_processes):
    schedule_counts.append([0]*number_of_timesteps)
    schedule_percentages.append([0]*number_of_timesteps)
    
for timestep in range(number_of_timesteps):
    scheduled_process = schedule[timestep]
    for process in range(number_of_processes):
        if timestep == 0:
            previous_count = 0
        else:
            previous_count = schedule_counts[process][timestep-1]
        schedule_counts[process][timestep] = previous_count
        if (process == scheduled_process):
            schedule_counts[process][timestep] = previous_count + 1
            
for timestep in range(number_of_timesteps):
    sum = 0
    for process in range(number_of_processes):
        sum = sum + schedule_counts[process][timestep]
    for process in range(number_of_processes):
        schedule_percentages[process][timestep] = schedule_counts[process][timestep] / (sum*1.0)

# Graph the summary
%matplotlib inline 
for process in range(number_of_processes):
    plotter.plot(schedule_percentages[process],label="Process " + process_names[process])
plotter.legend()


## Experimentation

Here are a few experiments you should try with this code:

* Explore smaller (`100`, `10`) and larger (`10000`) values for `number_of_timesteps` and see if the proportions converge to what you expect
* Add another process `D`, with an equal number of tickets as processes `A`,`B`, and `C`. Predict what the proportions should be in that case, and then run the simulation to check your prediction
* With the four processes in place (`A`,`B`,`C`,`D`), set the tickets so that process `A` obtains approximately 80% of the CPU time and the rest obtain an equal share of the remaining 20% of the CPU time. Run the simulation to verify if your settings work.